## Phantom Divide ($w=-1$)
Now I want to analyze how the equation of state parameter $w$ varies with different values of $w_0$, $w_a$, and redshift $z$. Specifically, the script aims to:

1. **Calculate $w$**:
   - Compute the equation of state parameter $w$ using the formula:
     $$
     w = w_0 + w_a \left(\frac{z}{1+z}\right)
     $$

2. **Determine the Distribution**:
   - Count the number of instances where $w$ is less than -1 and where $w$ is greater than -1 across different values of $w_0$, $w_a$, and redshift $z$.

3. **Identify Crossings**:
   - Find and record cases where $w$ transitions from being less than -1 to being greater than -1 (or vice versa) as the redshift $z$ varies.
   - Store these cases, including the values of $w_0$, $w_a$, the specific redshift where the transition occurs, and the values of $w$ before and after the transition.

4. **Output Results**:
   - Print the total counts of $w$ values that are less than -1 and greater than -1.
   - Output the minimum and maximum redshifts from the data file.
   - Print the specific $w_0$, $w_a$, and redshift values where $w$ crosses the -1 threshold.

The script uses these calculations to understand how $w$ behaves under different cosmological parameters and redshift values, with a particular focus on transitions across the $w = -1$ boundary.


In [1]:
import sys
import platform
import os
import pandas as pd
import numpy as np
import emcee
import corner
import matplotlib.pyplot as plt

_camb_path = os.path.join(os.path.dirname(os.path.abspath('.')), 'CAMB')
sys.path.insert(0, _camb_path)

import camb
print('Using CAMB %s installed at %s' % (camb.__version__, os.path.dirname(camb.__file__)))

# Load the data
file_path = 'lcparam_full_long.txt'

# Read the first line to get the header
with open(file_path, 'r') as file:
    header_line = file.readline().strip()
    header = header_line.lstrip('#').strip()

# Remove the last word from the header (assumed to be 'biascor')
header_words = header.split()
header_words.pop()
header = ' '.join(header_words)

# Read the rest of the file into a DataFrame
data = pd.read_csv(file_path, sep='\s+', skiprows=1, header=None)
data.columns = header.split()

z_data = data['zcmb'].values
z = np.linspace(np.min(z_data), np.max(z_data), 10)

# Initialize counters and storage
count_minus_one = 0
count_plus_one = 0
tot = 0

# Initialize a list to store results
crossings = []

# For loop that computes w for different values of w0 and wa at redshifts z
for w0 in np.linspace(-1.5, -0.5, 11):
    for wa in np.linspace(-0.5, 0.5, 11):
        previous_w = None
        for redshift in z:
            w = w0 + wa * (redshift / (1 + redshift))
            if previous_w is not None:
                if previous_w < -1 and w > -1:
                    crossings.append({'w0': w0, 'wa': wa, 'redshift': redshift, 'w_prev': previous_w, 'w_current': w})
                elif previous_w > -1 and w < -1:
                    crossings.append({'w0': w0, 'wa': wa, 'redshift': redshift, 'w_prev': previous_w, 'w_current': w})
            previous_w = w
            if w < -1:
                count_minus_one += 1
            if w > -1:
                count_plus_one += 1
            tot += 1

print(f"Number of w < -1: {count_minus_one}")
print(f"Number of w > -1: {count_plus_one}")

print(np.min(z_data), np.max(z_data))

w_max = -1.3 - 0.3 * (np.min(z_data) / (1 + np.min(z_data)))
w_min = -1.3 - 0.3 * (np.max(z_data) / (1 + np.max(z_data)))
print(w_max)
print(w_min)

# Print the crossings
print("Crossings of w <-1 and w >-1:")
for crossing in crossings:
    print(crossing)

Using CAMB 1.5.4 installed at /Users/marigalla/AstroCosmoComputational/Students/MariaLuisaGallavotti/CAMB/camb
Number of w < -1: 600
Number of w > -1: 600
0.01012 1.0608
-1.3030055834950303
-1.4544254658385094
Crossings of w <-1 and w >-1:
{'w0': -1.2, 'wa': 0.4, 'redshift': 1.0608, 'w_prev': -1.005755201605805, 'w_current': -0.9940993788819875}
{'w0': -1.2, 'wa': 0.5, 'redshift': 0.7105733333333334, 'w_prev': -1.0137095245000167, 'w_current': -0.9922996578145339}
{'w0': -1.1, 'wa': 0.20000000000000007, 'redshift': 1.0608, 'w_prev': -1.0028776008029026, 'w_current': -0.9970496894409938}
{'w0': -1.1, 'wa': 0.30000000000000004, 'redshift': 0.5938311111111112, 'w_prev': -1.0031021980171209, 'w_current': -0.9882257147000101}
{'w0': -1.1, 'wa': 0.4, 'redshift': 0.3603466666666667, 'w_prev': -1.0216456822640998, 'w_current': -0.994042694999314}
{'w0': -1.1, 'wa': 0.5, 'redshift': 0.3603466666666667, 'w_prev': -1.0020571028301246, 'w_current': -0.9675533687491424}
{'w0': -0.8999999999999999, 